# Api Twitter
* https://datascienceparichay.com/article/get-data-from-twitter-api-in-python-step-by-step-guide/
* https://developer.twitter.com/en/portal/projects/1391772372416532480/apps/20820453/keys

Análisis de sentimiento en Español:
* https://platzi.com/tutoriales/1874-python-lenguaje-natural/5654-realiza-un-analisis-de-sentimiento-en-3-pasos-con-python/
    * Ejemplo https://twitter.com/whaleandjaguar_?lang=en

In [1]:
import tweepy as tw
import getpass

In [2]:
my_api_key=getpass.getpass()

 ·························


In [4]:
my_api_secret=getpass.getpass()

 ··················································


In [5]:
auth = tw.OAuthHandler(my_api_key, my_api_secret)

In [6]:
api = tw.API(auth, wait_on_rate_limit=True)

In [9]:
search_query = "#SOSColomia OR #ParoNacional OR #Cali"

In [21]:
# get tweets from the API
tweets = tw.Cursor(api.search,
              q=search_query,
              lang="es",
              since="2021-03-01" #https://es.wikipedia.org/wiki/Protestas_en_Colombia_de_2021
                  ).items(5000) #empty for all 

In [22]:
# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

Total Tweets fetched: 5000


In [24]:
import pickle
f=open('tweets_copy.pickle','wb')
pickle.dump(tweets_copy,f)
f.close()

In [25]:
ls -lh tweets_copy.pickle

-rw-r--r-- 1 restrepo restrepo 25M May 12 01:05 tweets_copy.pickle


In [26]:
import pandas as pd

# intialize the dataframe
tweets_df = pd.DataFrame()

# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)

# show the dataframe
tweets_df.head()

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,Liliana Morales,,,False,2021-05-12 05:52:23,RT @SemanarioVOZ: El Semanario VOZ recalca su ...,None,Twitter for Android
1,Quique,"Bogotá, D.C., Colombia",Actitud positiva,False,2021-05-12 05:52:21,"RT @COMANDANTE_EJC: En #Cali, mi mensaje hacia...",[Cali],Twitter for Android
2,David Peña,,,False,2021-05-12 05:52:13,RT @photomauricio: En la décima jornada del Pa...,None,Twitter for Android
3,tulelegon,"Bogotá, D.C., Colombia",URIBISTA 💯 %,False,2021-05-12 05:52:12,RT @primeravoztv: Esto está pasando en estos m...,None,Twitter for iPhone
4,Aleyda Giraldo A,,,False,2021-05-12 05:52:09,RT @HRI_ONG: Vemos con preocupación las Fallas...,None,Twitter Web App


In [27]:
tweets_df

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,Liliana Morales,,,False,2021-05-12 05:52:23,RT @SemanarioVOZ: El Semanario VOZ recalca su ...,None,Twitter for Android
1,Quique,"Bogotá, D.C., Colombia",Actitud positiva,False,2021-05-12 05:52:21,"RT @COMANDANTE_EJC: En #Cali, mi mensaje hacia...",[Cali],Twitter for Android
2,David Peña,,,False,2021-05-12 05:52:13,RT @photomauricio: En la décima jornada del Pa...,None,Twitter for Android
3,tulelegon,"Bogotá, D.C., Colombia",URIBISTA 💯 %,False,2021-05-12 05:52:12,RT @primeravoztv: Esto está pasando en estos m...,None,Twitter for iPhone
4,Aleyda Giraldo A,,,False,2021-05-12 05:52:09,RT @HRI_ONG: Vemos con preocupación las Fallas...,None,Twitter Web App
...,...,...,...,...,...,...,...,...
4995,dcaroperez95,"Barranquilla, Colombia",Docente,False,2021-05-12 02:48:00,RT @ArielAnaliza: Reportando desde Cali.\n#Par...,[ParoNacional],Twitter Web App
4996,MANUEL MARCANO,VENEZUELA,Venezolano nacionalista. Considero que la mayo...,False,2021-05-12 02:47:59,RT @MyBolivia: 🔥 #URGENTE Otra noche de terror...,[URGENTE],Twitter for Android
4997,priveros🌻🌻👮‍♂️💚👮💚💚,,Siempre caminando por la derecha!! 🇨🇱🇨🇱🇨🇱,False,2021-05-12 02:47:58,RT @JavierOlivares: 🇨🇴 @IvanDuque viaja a #Cal...,[Cali],Twitter for Android
4998,tbz ♔: majo TOMBO QUE VEO TOMBO QUE PATEÓ,,b a n g t a n ☁️ s k z ☁️ t b z ☁️ e n h y p e...,False,2021-05-12 02:47:57,RT @val_10o: ⚠️ BOGOTÁ ⚠️ \n\nLas personas int...,[ParoNacional],Twitter Web App


In [34]:
tweets_df.to_json('tweets_df.json',orient='records')

In [31]:
set(tweets_df['hashtags'].dropna().sum())

{'10DeMayo',
 '10M',
 '10Mayo',
 '11M',
 '11May',
 '11Mayo',
 '12M',
 '12Mayo',
 '22N',
 '28a',
 'AEstaHora',
 'APararParaAvanzar',
 'ATENCION',
 'ATENCIÓN',
 'Actualidad',
 'AlAireMoradaNoticias',
 'Alerta',
 'AlertaRojaEnColombia',
 'AmericaTv',
 'Amigas',
 'Anonymous',
 'ApoyoAMiPolicia',
 'ApoyoAlEsmad',
 'AsambleaTwCol',
 'Asesinado',
 'Asesino',
 'Asesinos',
 'AsistenciaMilitarEs',
 'Atención',
 'Barranquilla',
 'BarrasUnidas',
 'BicicletadaPorLaVida',
 'Bielsa',
 'Bloqueos',
 'Bogota',
 'BogotaEnAlertaRoja',
 'Boyaca',
 'Bucaramanga',
 'BucaramangaResiste',
 'Buenaventura',
 'BuscarlosHastaEncontrarlos',
 'CALI',
 'CALIESTACENSURADO',
 'CALIRESISTENCIA',
 'CCC',
 'CNTIDenuncia',
 'COLOMBIA',
 'COVID19',
 'Caldas',
 'Caleña',
 'Cali',
 'CaliEnAlertaRoja',
 'CaliEnCensura',
 'CaliEnPeligro',
 'CaliEsResistencia',
 'CaliResiste',
 'CaliSOS',
 'CaliSeLevantaPorLaVida',
 'CaliSecuestrada',
 'Calico',
 'Calí',
 'Candelaria',
 'CaravanaHumanitaria',
 'CaribeAfirmativo',
 'Carmelo',
 'C